In [ ]:
!apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 1s (5,974 kB/s)
Selecting previously unselected package popp

In [ ]:
!pip install streamlit opencv-python-headless pytesseract pandas pdfplumber



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import pdfplumber
import os

def extract_tables_from_pdf(pdf_path):
    tables = []

    # Using pdfplumber to extract tables
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            # Extract tables using pdfplumber
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(pd.DataFrame(table))

    return tables

def ocr_extract_table_from_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    threshold_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 11, 2)

    # Find contours
    contours, _ = cv2.findContours(threshold_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    table_data = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        roi = image[y:y+h, x:x+w]

        # Use pytesseract to do OCR on the ROI
        text = pytesseract.image_to_string(roi, config='--psm 6')
        table_data.append(text)

    return table_data

def extract_tables_from_pdf_with_ocr(pdf_path):
    tables = []

    # Using pdfplumber to extract images from PDF
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            im = page.to_image()
            im_cv = np.array(im.original)
            table_data = ocr_extract_table_from_image(im_cv)
            tables.append(table_data)

    return tables

def process_multiple_pdfs(pdf_folder):
    # Get a list of PDF files in the specified folder
    pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

    # Iterate over each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)

        # Extract tables from the PDF using the provided functions
        tables = extract_tables_from_pdf(pdf_path)

        # Print or process the extracted tables as needed
        for table in tables:
            print(table)

# Example usage: Provide the folder containing the PDF files
pdf_folder = "/content/drive/MyDrive/FINANCIAL_DATA"
process_multiple_pdfs(pdf_folder)


Streaming output truncated to the last 5000 lines.
9           Profit / (Loss) for the year after income tax   
10      Other Comprehensive Income for the year before...   
11                Total Comprehensive Income for the year   
12                            Balance as at 31 March 2019   
13                Add: Impact on adoption of Ind AS 116 #   
14                           Balance as on 1st April 2019   
15                     Transfer to/from retained earnings   
16           Other Additions/Deductions during the year #   
17          Profit / (Loss) for the year after income tax   
18      Other Comprehensive Income for the year before...   
19                Total Comprehensive Income for the year   
20                            Balance as at 31 March 2020   

                       2                   3         4         5  \
0                   None                None      None      None   
1                                                                  
2   Reserves

In [ ]:
 process_multiple_pdfs(pdf_folder)


Streaming output truncated to the last 5000 lines.
9           Profit / (Loss) for the year after income tax   
10      Other Comprehensive Income for the year before...   
11                Total Comprehensive Income for the year   
12                            Balance as at 31 March 2019   
13                Add: Impact on adoption of Ind AS 116 #   
14                           Balance as on 1st April 2019   
15                     Transfer to/from retained earnings   
16           Other Additions/Deductions during the year #   
17          Profit / (Loss) for the year after income tax   
18      Other Comprehensive Income for the year before...   
19                Total Comprehensive Income for the year   
20                            Balance as at 31 March 2020   

                       2                   3         4         5  \
0                   None                None      None      None   
1                                                                  
2   Reserves

In [ ]:
 def combine_tables(tables):
    if not tables:
        return pd.DataFrame()
    combined_df = pd.concat(tables, ignore_index=True)
    return combined_df

In [ ]:
combine_tables(process_multiple_pdfs(pdf_folder))

Streaming output truncated to the last 5000 lines.
9           Profit / (Loss) for the year after income tax   
10      Other Comprehensive Income for the year before...   
11                Total Comprehensive Income for the year   
12                            Balance as at 31 March 2019   
13                Add: Impact on adoption of Ind AS 116 #   
14                           Balance as on 1st April 2019   
15                     Transfer to/from retained earnings   
16           Other Additions/Deductions during the year #   
17          Profit / (Loss) for the year after income tax   
18      Other Comprehensive Income for the year before...   
19                Total Comprehensive Income for the year   
20                            Balance as at 31 March 2020   

                       2                   3         4         5  \
0                   None                None      None      None   
1                                                                  
2   Reserves

""


In [ ]:
  # Specify the path to the output CSV file
output_csv = "/content/drive/MyDrive/FINANCIAL_DATA/tables.csv"

# Call the save_tables_to_csv function with the combined_df and output_csv arguments
save_tables_to_csv(combined_df, output_csv)

NameError: name 'save_tables_to_csv' is not defined

In [ ]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import pdfplumber
import os

def extract_tables_from_pdf(pdf_path):
    tables = []

    # Using pdfplumber to extract tables
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            # Extract tables using pdfplumber
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(pd.DataFrame(table))

    return tables

def ocr_extract_table_from_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    threshold_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 11, 2)

    # Find contours
    contours, _ = cv2.findContours(threshold_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    table_data = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        roi = image[y:y+h, x:x+w]

        # Use pytesseract to do OCR on the ROI
        text = pytesseract.image_to_string(roi, config='--psm 6')
        table_data.append(text)

    return table_data

def extract_tables_from_pdf_with_ocr(pdf_path):
    tables = []

    # Using pdfplumber to extract images from PDF
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            im = page.to_image()
            im_cv = np.array(im.original)
            table_data = ocr_extract_table_from_image(im_cv)
            tables.append(table_data)

    return tables

def process_multiple_pdfs(pdf_folder):
    # Get a list of PDF files in the specified folder
    pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

    all_tables = []
    # Iterate over each PDF file
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder, pdf_file)

        # Extract tables from the PDF using the provided functions
        tables = extract_tables_from_pdf(pdf_path)
        all_tables.extend(tables)

    # Convert all tables to a single DataFrame
    final_df = pd.concat(all_tables, axis=1)

    # Save DataFrame to a CSV file with the final name
    csv_file = "final_csv_files.csv"
    csv_path = os.path.join(pdf_folder, csv_file)
    final_df.to_csv(csv_path, index=False)

# Example usage: Provide the folder containing the PDF files
pdf_folder = "/content/drive/MyDrive/FINANCIAL_DATA"
process_multiple_pdfs(pdf_folder)
